In [1]:
import numpy as np

# Constants
k = 1.38e-16  # erg/K
mu = 4  # Mean molecular weight
m_u = 1.66e-24  # g
a = 7.56e-15  # erg/cm^3/K^4

#### Calculating Equations of State and Jacobian

def eos_pressure(rho, T):
    return (rho * k * T) / (mu * m_u) + (1 / 3) * a * T**4

def eos_energy(rho, T):
    return (3 / 2) * (k * T) / (mu * m_u) + (a * T**4) / rho

def jacobian(rho, T):
    dp_drho = k * T / (mu * m_u)
    dp_dT = (rho * k) / (mu * m_u) + (4 / 3) * a * T**3
    de_drho = -a * T**4 / rho**2
    de_dT = (3 / 2) * k / (mu * m_u) + (4 * a * T**3) / rho
    return np.array([[dp_drho, dp_dT], [de_drho, de_dT]])

def find_rho_T(p_star, e_star, rho_0, T_0, tol=1e-6, max_iter=100):
    rho, T = rho_0, T_0
    for _ in range(max_iter):
        p_0 = eos_pressure(rho, T)
        e_0 = eos_energy(rho, T)
        J = jacobian(rho, T)
        psi = np.array([p_0 - p_star, e_0 - e_star])
        
        # Solve J * delta = -psi
        delta = np.linalg.solve(J, -psi)
        
        # Update values
        rho += delta[0]
        T += delta[1]
        
        # Check convergence
        if np.linalg.norm(delta) < tol:
            break
    return rho, T

# Given values
p_star = 2.3e10  # erg/cm^3
e_star = 3.87e13  # erg/g
rho_0 = 1.0  # Initial guess for density 
T_0 = 1.0e7  # Initial guess for temperature 

rho, T = find_rho_T(p_star, e_star, rho_0, T_0)

# Verify
p_check = eos_pressure(rho, T)
e_check = eos_energy(rho, T)

print(f"Computed rho: {rho:.6e} g/cm^3")
print(f"Computed T: {T:.6e} K")
print(f"Check p: {p_check:.6e} erg/cm^3 (target: {p_star:.6e})")
print(f"Check e: {e_check:.6e} erg/g (target: {e_star:.6e})")


Computed rho: 9.883662e-04 g/cm^3
Computed T: 9.979947e+05 K
Check p: 2.300000e+10 erg/cm^3 (target: 2.300000e+10)
Check e: 3.870000e+13 erg/g (target: 3.870000e+13)
